# 네이버 지식인 크롤링

In [121]:
import time
import pandas as pd 
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.remote.webelement import WebElement
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
from urllib.parse import quote
from tqdm import tqdm
from datetime import datetime, timedelta

## 날짜 입력
- 며칠동안의 데이터를 수집할지 입력
- ex) 입력값: 3  ->  3일전 ~ 오늘날짜 

In [122]:
KEYWORD = '대선후보'
N = 3
start = datetime.today() - timedelta(N)
FROM_DATE = start.strftime("%Y.%m.%d.")
TO_DATE = datetime.today().strftime("%Y.%m.%d.")

In [124]:
print(FROM_DATE, TO_DATE)

2021.04.16. 2021.04.19.


In [125]:
driver = webdriver.Chrome('./chromedriver')
driver.maximize_window()
base_url = "https://www.naver.com/"
driver.get(base_url)
time.sleep(1)

# 지식iN 으로 이동
nav = driver.find_element_by_id('NM_FAVORITE')
lis = nav.find_element_by_tag_name('ul').find_elements_by_tag_name('li')
lis[3].click()

# 키워드 검색 후 최신순 정렬
search_area = driver.find_element_by_class_name('search_area')
search_area.find_element_by_tag_name('input').send_keys(KEYWORD)    # 키워드 입력
driver.find_element_by_class_name('search_btn').click()     # 검색
driver.find_element_by_class_name('option_sort').find_elements_by_tag_name('li')[1].click()     # '최신순' 클릭
time.sleep(1)

# 기간 검색: FROM_DATE ~ TO_DATE
driver.find_element_by_id('au_date_select').click()
time.sleep(1)
from_date = driver.find_element_by_id('sel_from_date')
to_date = driver.find_element_by_id('sel_to_date')
# 시작날짜, 완료날짜 입력
actions = ActionChains(driver).double_click(from_date).send_keys(FROM_DATE).double_click(to_date).send_keys(TO_DATE)
actions.perform()
driver.find_element_by_xpath('/html/body/div[3]/div/ul/li[4]/input[3]').click()

# 전체 페이지수 수집
while True:
    pagination = driver.find_element_by_class_name('s_paging')
    pages = pagination.find_elements_by_tag_name('a')
    tmp = []
    for page in pages:
        tmp.append(page.text)
    if '다음페이지' in tmp:
        pages[-1].click()
        time.sleep(2)
    else:
        NUM_OF_PAGE = int(pagination.text.split()[-1])
        break
print("전체 페이지 수: ", NUM_OF_PAGE)
driver.close()

전체 페이지 수:  3


In [126]:
driver = webdriver.Chrome('./chromedriver')
driver.maximize_window()
ans_url_list = []; total_ans = 0

# 질문들의 url 수집
for page in range(NUM_OF_PAGE):
    kin_url = f'https://kin.naver.com/search/list.nhn?sort=date&query={quote(KEYWORD)}'
    period_url = f'&period={FROM_DATE}%7C{TO_DATE}&section=kin'
    page_url = f'&page={page+1}'
    url = kin_url + period_url + page_url
    driver.get(url)
    time.sleep(1)
    #print(f"{page+1} 페이지")

    ul = driver.find_element_by_xpath('//*[@id="s_content"]/div[3]/ul')
    lis = ul.find_elements_by_tag_name('li')
    for li in lis:
        a_tag = li.find_element_by_tag_name('a')
        ans_url_list.append(a_tag.get_attribute('href'))

print(f'총 {len(ans_url_list)}개의 질문')

# 각 질문의 답변 개수
for i, ans_url in enumerate(ans_url_list):
    driver.get(ans_url)
    time.sleep(0.5)
    ans_cnt = int(driver.find_element_by_class_name('_answerCount.num').text)
    total_ans += ans_cnt
    print(f'{i+1}번째 : {ans_cnt}')
print("----------------------------------")
print(f'Total 답변 개수: {total_ans}')

driver.close()

총 23개의 질문
1번째 : 1
2번째 : 1
3번째 : 3
4번째 : 3
5번째 : 3
6번째 : 5
7번째 : 3
8번째 : 1
9번째 : 2
10번째 : 4
11번째 : 1
12번째 : 6
13번째 : 6
14번째 : 1
15번째 : 1
16번째 : 3
17번째 : 1
18번째 : 1
19번째 : 5
20번째 : 2
21번째 : 4
22번째 : 2
23번째 : 12
----------------------------------
Total 답변 개수: 71


In [127]:
driver = webdriver.Chrome('./chromedriver')
driver.maximize_window()
answer_text_list = []

for ans_url in tqdm(ans_url_list):
    driver.get(ans_url)
    time.sleep(1)
    more = driver.find_element_by_xpath('//*[@id="nextPageButton"]')
    disp = more.get_attribute('style')      # 더보기 할 답변이 남아있지 않은 경우 "style:None" 이 존재한다
    try:
        # 더보기 할 답변이 남지 않을 때까지 while문을 돌린다
        while disp:     
            driver.find_element_by_id('nextPageButton').click()
            time.sleep(1)
    except:
        time.sleep(1)
    answers = driver.find_elements_by_class_name('answer-content__item._contentWrap._answer')
    for ans in answers:
        ps = ans.find_elements_by_tag_name('p')
        for p in ps:
            answer_text_list.append(p.text)
driver.close()

100%|██████████| 23/23 [01:17<00:00,  3.35s/it]


In [128]:
len(answer_text_list)

1544

### 원본 저장(전처리 X)

In [129]:
df = pd.DataFrame({
    'answer': answer_text_list
})
df.head()

,answer
0,가린샤 님 답변
1,먼저 이성윤 중앙지검장이 수사저지가 적법했는지를 살펴보아야할 것 같아요.
2,
3,그리고 난후에 검찰 수사가 적법했는지 보아야지 앞뒤가 맞는 것 같아요.
4,


In [130]:
df.to_csv('./data/answer_origin_test.txt', sep='\t', index=False)

### 데이터 전처리

In [131]:
df_test = pd.read_table('./data/answer_origin_test.txt')
df_test.head()

,answer
0,가린샤 님 답변
1,먼저 이성윤 중앙지검장이 수사저지가 적법했는지를 살펴보아야할 것 같아요.
2,NaN
3,그리고 난후에 검찰 수사가 적법했는지 보아야지 앞뒤가 맞는 것 같아요.
4,NaN


In [132]:
df_test.shape

(1542, 1)

In [133]:
# 공백 여부 확인
drop_idx = df_test[df_test.answer == ""].index
print(len(drop_idx), drop_idx)

0 Int64Index([], dtype='int64')


In [134]:
# null값 여부 확인
df_test.isnull().sum()

answer    429
dtype: int64

In [135]:
# null값을 가지고 있는 행 제거
df_test.dropna(axis=0, inplace=True)
df_test.shape

(1113, 1)

In [136]:
df_test.head()

,answer
0,가린샤 님 답변
1,먼저 이성윤 중앙지검장이 수사저지가 적법했는지를 살펴보아야할 것 같아요.
3,그리고 난후에 검찰 수사가 적법했는지 보아야지 앞뒤가 맞는 것 같아요.
5,계속 수사 거부하다가 이제야 와서 수사를 받고 또 그 내용을 다 공개하면
7,이것은 검찰로써는 상당히 부끄러운 이야기인거죠.


In [137]:
print(f'전처리 후 샘플의 개수: {len(df_test)}')

전처리 후 샘플의 개수: 1113


In [138]:
df_test.to_csv('./data/answer_result_test.txt', sep='\t', index=False)